# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests  #http request, execute API
# import xlsxwriter
from google.colab import drive
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
url ='https://github.com/jackgurae/algo_trading_fcc/blob/515b8bd68bbd4ecef55fa1984954c013af68be15/sp_500_stocks.csv'
url = '/content/drive/MyDrive/Cloud/algo_trading/sp_500_stocks.csv'
stocks = pd.read_csv(url, sep=',', delimiter=None)

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [7]:
# from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [16]:
api = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={api}'
data = requests.get(api_url).json()
# print(data.status_code)
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'G TLA/K)NNOA(GEETSQLELARMS C BDSA', 'calculationPrice': 'close', 'open': 127.12, 'openTime': 1657752029904, 'openSource': 'lcifafoi', 'close': 125.16, 'closeTime': 1679765355360, 'closeSource': 'fafocili', 'high': 128.23, 'highTime': 1641567242890, 'highSource': 'reaiyeecpd euidmt l1n 5', 'low': 125.16, 'lowTime': 1674644591601, 'lowSource': 'eapeey 5clt niurdmied1 ', 'latestPrice': 127.03, 'latestSource': 'Close', 'latestTime': 'May 28, 2021', 'latestUpdate': 1683877308622, 'latestVolume': 71713954, 'iexRealtimePrice': 129.38, 'iexRealtimeSize': 2702, 'iexLastUpdated': 1671421599884, 'delayedPrice': 124.93, 'delayedPriceTime': 1672621685267, 'oddLotDelayedPrice': 130.65, 'oddLotDelayedPriceTime': 1642116306478, 'extendedPrice': 125.89, 'extendedChange': 0.01, 'extendedChangePercent': 8e-05, 'extendedPriceTime': 1678997773199, 'previousClose': 130.05, 'previousVolume': 98616718, 'change': -0.68, 'changePercent': -0.0054

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [20]:
price = data['close']
market_cap = data['marketCap']
print(price)
print(market_cap)

125.16
2170032023292


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [21]:
my_columns = ['Ticker', 'Stock Price', 'Market Cap', '# of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Cap,# of Shares to Buy


In [26]:
final_dataframe.append(
  pd.Series(
      [
        symbol,
        price,
        market_cap,
        'N/A'
      ], index=my_columns
  ), ignore_index = True
)

,Ticker,Stock Price,Market Cap,# of Shares to Buy
0,AAPL,125.16,2170032023292,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [36]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
  api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={api}'
  data = requests.get(api_url).json()
  final_dataframe = final_dataframe.append(
      pd.Series(
          [
           stock,
           data['close'],
           data['marketCap'],
           'N/A'
          ], index = my_columns
      ), ignore_index = True
  )

In [37]:
final_dataframe

,Ticker,Stock Price,Market Cap,# of Shares to Buy
0,A,140.71,42454614156,N/A
1,AAL,24.41,15795365512,N/A
2,AAP,198.25,12505616607,N/A
3,AAPL,125.86,2151755238677,N/A
4,ABBV,116.60,200449528297,N/A
...,...,...,...,...
500,YUM,123.67,36454773432,N/A
501,ZBH,174.41,36642752191,N/A
502,ZBRA,518.88,26914778831,N/A
503,ZION,60.00,9543187811,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [39]:
def chunks(lst, n):
  for i in range(0, len(lst), n):
    yield lst[i: i + n]

In [57]:
symbol_g = list(chunks(stocks['Ticker'], 100))
symbol_str = []
type(symbol_g)
for i in range(0, len(symbol_g)):
  symbol_str.append(','.join(symbol_g[i]))
  # print(symbol_str[i])
final_dataframe = pd.DataFrame(columns=my_columns)

for symbols in symbol_str:
  batch_api_call_url =f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=quote&token={api}'
  # print(batch_api_call_url)
  data = requests.get(batch_api_call_url).json()

  for symbol in symbols.split(','):
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
              symbol,
              data[symbol]['quote']['close'],
              data[symbol]['quote']['marketCap'],
              'N/A'
            ], index = my_columns
        ), ignore_index=True
    )
final_dataframe

,Ticker,Stock Price,Market Cap,# of Shares to Buy
0,A,141.69,43170005122,N/A
1,AAL,24.54,16122497801,N/A
2,AAP,195.79,12484724454,N/A
3,AAPL,130.56,2105133627583,N/A
4,ABBV,117.90,206132859528,N/A
...,...,...,...,...
500,YUM,121.05,36690124574,N/A
501,ZBH,173.89,35943837481,N/A
502,ZBRA,517.85,27767475548,N/A
503,ZION,58.88,9574555499,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: